In [1]:
from auramask.metrics import EmbeddingDistance, PerceptualSimilarity
from auramask.losses import FaceEmbedEnum

2024-02-16 18:23:13.928880: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-16 18:23:13.965252: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 18:23:13.965275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 18:23:13.966160: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 18:23:13.972125: I tensorflow/core/platform/cpu_feature_guar

Using TensorFlow backend


/opt/miniconda/envs/unet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Inputs

### Victim Models (F)

In [2]:
F = [FaceEmbedEnum.ARCFACE, FaceEmbedEnum.DEEPFACE, FaceEmbedEnum.DEEPID]
F_set = FaceEmbedEnum.build_F(F)

2024-02-16 18:23:17.053224: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 18:23:17.088826: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-16 18:23:17.090277: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### HyperParams

In [3]:
lda = 0.2
# Backbones are 'vgg', 'alex', 'squeeze'
backbone = 'vgg'

# Metrics

## Perceptual Similarity

In [4]:
lpips = PerceptualSimilarity(backbone=backbone, spatial=False, l=lda)

In [5]:
lpips.get_config()

{'name': 'PerceptualLoss', 'backbone': 'vgg', 'spatial': False, 'model': {}}

## Embedding Distance

In [6]:
embeddist = EmbeddingDistance(F=F)

In [7]:
embeddist.get_config()

{'name': 'EmbeddingsLoss',
 'Cosine Similarity': <keras.src.metrics.regression_metrics.CosineSimilarity at 0x7f2de1028f50>}

# Test
Testing the giving parts of the loss

In [8]:
import tensorflow_datasets as tfds
from keras_cv.layers import Resizing, Rescaling, Augmenter, RandomContrast, RandomColorDegeneration
from tensorflow.data import AUTOTUNE
BATCH_SIZE = 16

In [9]:
ds, info = tfds.load('lfw',
                     decoders=tfds.decode.PartialDecoding({
                       'image': True,
                     }),
                     with_info=True,
                     download=True,
                     as_supervised=False)

In [10]:
train_ds = ds['train']

In [11]:
SEED = int.from_bytes("Pochita Lives".encode())

augmenter = Augmenter(
  [
    Rescaling(1./255),
    Resizing(128,128),
  ]
)

noise_addition = Augmenter(
  [
    RandomContrast([0.0,1.0], [-0.5,0.5], seed=SEED),
    RandomColorDegeneration([0,0.5], seed=SEED),
  ]
)

def preprocess_data(images):
  inputs = {"images": images}
  outputs_1 = augmenter(inputs)
  outputs_2 = noise_addition(inputs)
  return outputs_1['images'], outputs_2['images']

In [12]:
train_ds = train_ds.batch(BATCH_SIZE).map(
  lambda x: preprocess_data(x['image']),
  num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

## Metrics Computation

In [13]:
for batch in train_ds.take(5):
  dist = embeddist(batch[0], batch[1])
  perceptual = lpips(batch[0], batch[1])
  print("Metrics:\n", "\tF-Dist: %0.3f\n"%(dist), "\tPerceptual: %0.3f\n"%(perceptual))

2024-02-16 18:23:38.967160: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-16 18:23:39.025695: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-16 18:23:39.333963: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Metrics:
 	F-Dist: -0.757
 	Perceptual: 0.113

Metrics:
 	F-Dist: -1.056
 	Perceptual: 0.119

Metrics:
 	F-Dist: -1.171
 	Perceptual: 0.117

Metrics:
 	F-Dist: -1.206
 	Perceptual: 0.119



2024-02-16 18:23:54.863490: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Metrics:
 	F-Dist: -1.220
 	Perceptual: 0.116

